In [ ]:
import pandas as pd
import numpy as np
#import seaborn as sns
#import matplotlib.pyplot as plt
import re

from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer 
#from sklearn.model_selection import cross_val_score, train_test_split 
#from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
# read in the dataframe

df = pd.read_csv('../data/cleaned/combined.csv')

In [3]:
# check the shape and the first 5 rows

print(df.shape)
df.head()

(220671, 260)


,subreddit,author,date,post,tfidf_abl,tfidf_abus,tfidf_actual,tfidf_addict,tfidf_adhd,tfidf_advic,...,tfidf_wish,tfidf_without,tfidf_wonder,tfidf_work,tfidf_worri,tfidf_wors,tfidf_would,tfidf_wrong,tfidf_x200b,tfidf_year
0,alcoholism,RobynTacoo,2019/10/19,My husband offered me a glass of wine I turned...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.479482,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,alcoholism,wolsinyourarea,2019/10/19,Feeling like a failure Broke my 2 week streak ...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,alcoholism,glitterONeverything,2019/10/19,help! withdrawals are crazy!! I feel lije shit...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,alcoholism,Collector420,2019/10/19,God damn I (M17) wrote messages to almost 20 p...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,alcoholism,engineerkoala,2019/10/19,How to have as much fun as before Today was my...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.337441,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Feature Engineering and Preprocessing

In [4]:
## count sentences in each tweet (row)

# set up an emty list for sentence counts

n_sentence = []

# iterate through the text column

for i in range(len(df['post'])):
    
    n_sentence.append(len(sent_tokenize(df['post'][i].lower())))
    
# store it in the dataframe

df['n_sentence'] = n_sentence

In [5]:
# instantiate lemmatizer and tokenizer

lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+|\$[\d\.]+|\S+')

In [7]:
## tokenize and lemmatize

# create a column 'text_clean'

df['post_clean'] = ''

# iterate through each row in the column text_all
    
for i in range(len(df['post'])):
        
    # tokenize each word in text into its own string
    post_token = []
    post_token.extend(tokenizer.tokenize(df['post'][i].lower()))
    post_tokens = []
    [post_tokens.append(word) for word in post_token if word not in post_tokens]
        
    # lemmatize the words
    post_lemmatize = []
    for j in range(len(post_tokens)):
        post_lemmatize.append(lemmatizer.lemmatize(post_tokens[j]))
        
    # remove characters and numbers
    clean_post = []
    for k in range(len(post_lemmatize)):
        clean_post.append(re.sub('[^a-zA-Z]', '', post_lemmatize[k]))    
        
    # group them together
    post_collection = [post for post in clean_post]
    
    # put the words back to one long string
    post_collection = ' '.join(post_collection)

    # fill new column
    df['post_clean'][i] = post_collection

/Users/juhee/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [12]:
## count words in each tweet (row)

# set up an empty list for word counts

n_words = []

# iterate through the text column

for i in range(len(df['post_clean'])):
    
    word_tokens = tokenizer.tokenize(df['post_clean'][i])
    cnt = len(word_tokens)
    n_words.append(cnt)
    
# store it in the dataframe

df['n_words'] = n_words

In [9]:
# check to see if the columns have been created

df[['post', 'post_clean', 'n_sentence', 'n_words']]

,post,post_clean,n_sentence,n_words
0,My husband offered me a glass of wine I turned...,my husband offered me a glass of wine i turned...,2,17
1,Feeling like a failure Broke my 2 week streak ...,feeling like a failure broke my week streak a...,3,23
2,help! withdrawals are crazy!! I feel lije shit...,help withdrawal are crazy i feel lije shit a...,7,48
3,God damn I (M17) wrote messages to almost 20 p...,god damn i m wrote message to almost people b...,2,66
4,How to have as much fun as before Today was my...,how to have a much fun before today wa my seco...,5,62
...,...,...,...,...
220666,My worst fear came to life today. One of my fr...,my worst fear came to life today one of frien...,6,87
220667,***GROSS ANXIETY STORY*** help... This is real...,gross anxiety story help this is really gros...,7,108
220668,Really need advice and help please The last mo...,really need advice and help please the last mo...,1,60
220669,Anxiety to live up in the tech industry I’m a ...,anxiety to live up in the tech industry i m a ...,24,207


In [13]:
## count each character without white space in each tweet (row) to calculate readability index

# set up an empty list for character counts

n_chars = []

# iterate through the text column

for i in range(len(df['post_clean'])):
    
    n_chars.append(len(df['post_clean'][i].strip()))
    
# store it in the dataframe

df['n_chars'] = n_chars

In [16]:
df['n_chars'][:5]

0     88
1    122
2    250
3    343
4    331
Name: n_chars, dtype: int64

In [19]:
# https://www.webfx.com/tools/read-able/automated-readability-index.html#:~:text=Readability%20indices%20also%20take%20into,words%2Fsentences)%20%2D%2021.43.
# calculate automated readability index 

df['ari'] = round((4.71 * (df['n_chars']/df['n_words']) + 0.5 * (df['n_words']/df['n_sentence']) - 21.43), 2)

In [20]:
df['ari'][:5]

0     7.20
1     7.39
2     6.53
3    19.55
4     9.92
Name: ari, dtype: float64

In [21]:
# save the final file to csv

df.to_csv('../data/cleaned/final.csv', index = False)